# SDP relaxations for the CHSH game

In [88]:
import lib_non_local_games as nlg
import numpy as np
import cvxpy as cp
import functools as fc
from operator import mul

In [89]:
cp.installed_solvers()

['CVXOPT', 'ECOS', 'ECOS_BB', 'GLPK', 'GLPK_MI', 'MOSEK', 'OSQP', 'SCS']

Parameter of the CHSH game

In [90]:
dimA1 = 2
dimA2 = 2
dimQ1 = 2
dimQ2 = 2

dimT = 2
dimS = 2

probQ1 = (.5,.5)
probQ2 = (.5,.5)

# Symmetry reduction of CHSH game $n_1 = 1$, $n_2 = 1$.

Here we make use of the symmetry reduction for the first level of the hierarchy of relaxations.

NOTE: I think I have now found all the constraints but you never know, maybe there are more... The idea is that you should take the four operators/distributions we have and see what happen when the constraints on rho are applied.

In [108]:
# Subsystems A1 Q1 A2 Q2
subs_A1Q1A2Q2 = (dimA1,dimQ1,dimA2,dimQ2)
indices_A1Q1A2Q2 = nlg.indices_list(subs_A1Q1A2Q2)
dim_A1Q1A2Q2 = fc.reduce(mul, subs_A1Q1A2Q2, 1)

# Subsystems A2 Q1 Q2
subs_A2Q1Q2 = (dimA2,dimQ1,dimQ2)
indices_A2Q1Q2 = nlg.indices_list(subs_A2Q1Q2)

# Subsystems A1 Q1 Q2
subs_A1Q1Q2 = (dimA1,dimQ1,dimQ2)
indices_A1Q1Q2 = nlg.indices_list(subs_A1Q1Q2)

# Subsystems A1 Q1
subs_A1Q1 = (dimA1,dimQ1)
indices_A1Q1 = nlg.indices_list(subs_A1Q1)
dim_A1Q1 = fc.reduce(mul, subs_A1Q1, 1)

# Subsystems A2 Q2
subs_A2Q2 = (dimA2,dimQ2)
indices_A2Q2 = nlg.indices_list(subs_A2Q2)
dim_A2Q2 = fc.reduce(mul, subs_A2Q2, 1)

# Subsystems A1 A2
subs_A1A2 = (dimA1,dimA2)
indices_A1A2 = nlg.indices_list(subs_A1A2)

# Subsystems Q1 Q2
subs_Q1Q2 = (dimQ1,dimQ2)
indices_Q1Q2 = nlg.indices_list(subs_Q1Q2)

# Subsystems T hat(T) S hat(S)
subs_TTSS = (dimT,dimT,dimS,dimS)
indices_TTSS = nlg.indices_list(subs_TTSS)
dim_TTSS = fc.reduce(mul, subs_TTSS, 1)

# Additional dimensions
dim_TSS = dimT * dimS**2
dim_TT = dimT**2
dim_SS = dimS**2

# State on subsystem T
rhoT = np.identity(dimT)/dimT

The program

In [92]:
random_rule = np.random.randint(0,2,dim_A1Q1A2Q2)

def random_rule_function_A1Q1A2Q2(a1,q1,a2,q2):
    return( random_rule[nlg.binarytoint([a1,q1,a2,q2])] )

def allwin_rule_function_A1Q1A2Q2(a1,q1,a2,q2):
    return( 1 )

In [93]:
print([random_rule_function_A1Q1A2Q2(*index) for index in indices_A1Q1A2Q2])

[0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1]


In [33]:
## VARIABLES

# The (sub-normalized) probability distributions we optimize over
p_A1Q1A2Q2 = cp.Variable(dim_A1Q1A2Q2,nonneg=True)
q_A1Q1A2Q2 = cp.Variable(dim_A1Q1A2Q2,nonneg=True)
r_A1Q1A2Q2 = cp.Variable(dim_A1Q1A2Q2)
s_A1Q1A2Q2 = cp.Variable(dim_A1Q1A2Q2)

## OBJECTIVE FUNCTION

# The rule function V(a1,q1,a2,q2)
#V = nlg.CHSH_rule_function_A1Q1A2Q2
#V = allwin_rule_function_A1Q1A2Q2
V = random_rule_function_A1Q1A2Q2

# The object function is
obj_fun_components = [V(*index) * ( p_A1Q1A2Q2[nlg.binarytoint(index)] - q_A1Q1A2Q2[nlg.binarytoint(index)] )
                      for index in indices_A1Q1A2Q2]

object_function = cp.Constant(dimT**2) * sum(obj_fun_components)

## CONSTRAINTS

constraints = []
    
# 1) Positivity
constraints.append( p_A1Q1A2Q2 + ( r_A1Q1A2Q2 + s_A1Q1A2Q2 )/2. >= 0 )
constraints.append( p_A1Q1A2Q2 - ( r_A1Q1A2Q2 + s_A1Q1A2Q2 )/2. >= 0 )
constraints.append( q_A1Q1A2Q2 + ( r_A1Q1A2Q2 - s_A1Q1A2Q2 )/2. >= 0 )
constraints.append( q_A1Q1A2Q2 - ( r_A1Q1A2Q2 - s_A1Q1A2Q2 )/2. >= 0 )


# 2) Trace = 1
norm_p_q_plus = sum([p_A1Q1A2Q2[i] + q_A1Q1A2Q2[i] for i in map(nlg.binarytoint,indices_A1Q1A2Q2)])
norm_p_q_minus = sum([p_A1Q1A2Q2[i] - q_A1Q1A2Q2[i] for i in map(nlg.binarytoint,indices_A1Q1A2Q2)])
norm_r = sum([r_A1Q1A2Q2[i] for i in map(nlg.binarytoint,indices_A1Q1A2Q2)])
norm_s = sum([s_A1Q1A2Q2[i] for i in map(nlg.binarytoint,indices_A1Q1A2Q2)])

constraints.append( norm_p_q_plus - 1 == 0 )
constraints.append( norm_p_q_minus + 1 >= 0 )
constraints.append( norm_p_q_minus - 1 <= 0 )
constraints.append( norm_r + 1 >= 0 )
constraints.append( norm_r - 1 <= 0 )
constraints.append( norm_s + 1 >= 0 )
constraints.append( norm_s - 1 <= 0 )

# 2) First linear constraint
for a2,q1,q2 in indices_A2Q1Q2:
    indices_A1q1a2q2 = [nlg.binarytoint([a,q1,a2,q2]) for a in range(dimA1)]
    indices_A1Q1a2q2 = [nlg.binarytoint([a,q,a2,q2]) for a,q in indices_A1Q1]
    
    # p + q
    lhs_1 = sum([p_A1Q1A2Q2[i]+q_A1Q1A2Q2[i] for i in indices_A1q1a2q2])
    rhs_1 = cp.Constant(probQ1[q1]) * sum([p_A1Q1A2Q2[i]+q_A1Q1A2Q2[i] for i in indices_A1Q1a2q2])
    
    # p - q
    lhs_2 = sum([p_A1Q1A2Q2[i]-q_A1Q1A2Q2[i] for i in indices_A1q1a2q2])
    rhs_2 = cp.Constant(probQ1[q1]/dimT) * sum([r_A1Q1A2Q2[i] for i in indices_A1Q1a2q2])
    
    # r
    lhs_3 = sum([r_A1Q1A2Q2[i] for i in indices_A1q1a2q2])
    rhs_3 = cp.Constant(probQ1[q1]) * sum([r_A1Q1A2Q2[i] for i in indices_A1Q1a2q2])
    
    # s
    lhs_4 = sum([s_A1Q1A2Q2[i] for i in indices_A1q1a2q2])
    rhs_4 = cp.Constant(probQ1[q1]/dimT) * sum([p_A1Q1A2Q2[i]+q_A1Q1A2Q2[i] for i in indices_A1Q1a2q2])
    
    constraints.append( lhs_1 - rhs_1 == 0 )
    constraints.append( lhs_2 - rhs_2 == 0 )
    constraints.append( lhs_3 - rhs_3 == 0 )
    constraints.append( lhs_4 - rhs_4 == 0 )

# 3) Second linear constraint
for a1,q1,q2 in indices_A1Q1Q2:
    indices_a1q1A2q2 = [nlg.binarytoint([a1,q1,a,q2]) for a in range(dimA2)]
    indices_a1q1A2Q2 = [nlg.binarytoint([a1,q1,a,q]) for a,q in indices_A2Q2]
    
    # p + q
    lhs_1 = sum([p_A1Q1A2Q2[i]+q_A1Q1A2Q2[i] for i in indices_a1q1A2q2])
    rhs_1 = cp.Constant(probQ2[q2]) * sum([p_A1Q1A2Q2[i]+q_A1Q1A2Q2[i] for i in indices_a1q1A2Q2])
    
    # p - q
    lhs_2 = sum([p_A1Q1A2Q2[i]-q_A1Q1A2Q2[i] for i in indices_a1q1A2q2])
    rhs_2 = cp.Constant(probQ2[q2]/dimT) * sum([s_A1Q1A2Q2[i] for i in indices_a1q1A2Q2])
    
    # r
    lhs_3 = sum([r_A1Q1A2Q2[i] for i in indices_a1q1A2q2])
    rhs_3 = cp.Constant(probQ2[q2]/dimT) * sum([p_A1Q1A2Q2[i]+q_A1Q1A2Q2[i] for i in indices_a1q1A2Q2])
    
    # s
    lhs_4 = sum([s_A1Q1A2Q2[i] for i in indices_a1q1A2q2])
    rhs_4 = cp.Constant(probQ2[q2]) * sum([s_A1Q1A2Q2[i] for i in indices_a1q1A2Q2])
    
    constraints.append( lhs_1 - rhs_1 == 0 )
    constraints.append( lhs_2 - rhs_2 == 0 )
    constraints.append( lhs_3 - rhs_3 == 0 )
    constraints.append( lhs_4 - rhs_4 == 0 )
        
# 4) PPT conditions
constraints.append( dimT * (p_A1Q1A2Q2 + q_A1Q1A2Q2) - r_A1Q1A2Q2 >= 0 )
constraints.append( dimT * (p_A1Q1A2Q2 + q_A1Q1A2Q2) - s_A1Q1A2Q2 >= 0 )
constraints.append( dimT * r_A1Q1A2Q2 - (p_A1Q1A2Q2 - q_A1Q1A2Q2) >= 0 )
constraints.append( dimT * s_A1Q1A2Q2 - (p_A1Q1A2Q2 - q_A1Q1A2Q2) >= 0 )
constraints.append( p_A1Q1A2Q2 - q_A1Q1A2Q2 >= 0 )
constraints.append( r_A1Q1A2Q2 >= 0 )
constraints.append( s_A1Q1A2Q2 >= 0 )

# Write the problem
prob = cp.Problem(cp.Maximize(object_function), constraints)

In [34]:
prob.solve(verbose=True)

-----------------------------------------------------------------
           OSQP v0.5.0  -  Operator Splitting QP Solver
              (c) Bartolomeo Stellato,  Goran Banjac
        University of Oxford  -  Stanford University 2018
-----------------------------------------------------------------
problem:  variables n = 64, constraints m = 279
          nnz(P) + nnz(A) = 1120
settings: linear system solver = qdldl,
          eps_abs = 1.0e-05, eps_rel = 1.0e-05,
          eps_prim_inf = 1.0e-04, eps_dual_inf = 1.0e-04,
          rho = 1.00e-01 (adaptive),
          sigma = 1.00e-06, alpha = 1.60, max_iter = 10000
          check_termination: on (interval 25),
          scaling: on, scaled_termination: off
          warm start: on, polish: on

objective    pri res    dua res    rho        time
   1  -1.7874e+01   1.47e+00   5.68e+02   1.00e-01   4.39e-04s
 200  -7.5241e-01   5.72e-04   2.17e-03   5.98e-01   1.26e-03s
 375  -7.4995e-01   1.22e-05   1.75e-05   4.94e-01   6.95e-04s

statu

0.7499470975862842

Unnormalized maximally-entangled state, for the objective function

In [94]:
# Maximally entangled vectors between T|S and TT|SS
phi_TS = nlg.bipartite_unnorm_max_entangled_state(dimT)
phi_TSTS = nlg.tensor([phi_TS,phi_TS])

# Maximally mixed states between TT|SS (correct order of subsystems)
Phi_TSTS = np.outer(phi_TSTS,phi_TSTS)
P = nlg.permutation_matrix((0,1,2,3), (0,2,1,3), (dimT, dimT,dimT, dimT))
Phi_TTSS = P @ Phi_TSTS @ P

The program, with additional NPA constraints

Still not enough! Is there a error in the new constraint, or we miss some additional constraints?

In [110]:
## VARIABLES

# The (sub-normalized) states we optimize over
rho_TTSS = []
for i in map(nlg.binarytoint,indices_A1Q1A2Q2):
    rho_TTSS.append( cp.Variable((dim_TTSS,dim_TTSS),hermitian=True) )

## OBJECTIVE FUNCTION

# The rule function is
rule_A1Q1A2Q2 = nlg.CHSH_rule_function_A1Q1A2Q2
#rule_A1Q1A2Q2 = random_rule_function_A1Q1A2Q2

# The object function is
obj_fun_components = [rule_A1Q1A2Q2(*index) * cp.trace( cp.matmul(Phi_TTSS,rho_TTSS[nlg.binarytoint(index)]) )
                      for index in indices_A1Q1A2Q2]

object_function = cp.Constant(dimT**2) * sum(obj_fun_components)

## CONSTRAINTS

constraints = []
    
# 2) rho_TTSS are (sub-normalized) quantum states
# 2a) trace of the sum is 1
constraints.append( sum([cp.trace(rho_TTSS[i]) for i in map(nlg.binarytoint,indices_A1Q1A2Q2)]) - 1 == 0 )

# 2b) positive semidefinite matrices
for i in map(nlg.binarytoint,indices_A1Q1A2Q2):
    constraints.append( rho_TTSS[i] >> 0 )
    
# 3) First linear constraint
for a2,q1,q2 in indices_A2Q1Q2:
    indices_A1q1a2q2 = [nlg.binarytoint([a,q1,a2,q2]) for a in range(dimA1)]
    indices_A1Q1a2q2 = [nlg.binarytoint([a,q,a2,q2]) for a,q in indices_A1Q1]
    
    lhs = sum([rho_TTSS[i] for i in indices_A1q1a2q2])
    
    rhs_variable = sum([rho_TTSS[i] for i in indices_A1Q1a2q2])
    rhs_partial = nlg.partial_trace(rhs_variable, [dimT, dim_TSS])
    rhs = cp.Constant(probQ1[q1]) * cp.kron(rhoT, rhs_partial)
    
    constraints.append( lhs - rhs == 0 )
    
# 4) Second linear constraint
P = nlg.permutation_matrix((0,1,2,3),(1,0,2,3),subs_TTSS)

for a1,q1,q2 in indices_A1Q1Q2:
    indices_a1q1A2q2 = [nlg.binarytoint([a1,q1,a,q2]) for a in range(dimA2)]
    indices_a1q1A2Q2 = [nlg.binarytoint([a1,q1,a,q]) for a,q in indices_A2Q2]
    
    lhs_variable = sum([rho_TTSS[i] for i in indices_a1q1A2q2])
    lhs = cp.matmul( cp.matmul(P,lhs_variable) , P )
    
    rhs_variable = sum([rho_TTSS[i] for i in indices_a1q1A2Q2])
    rhs_permuted = cp.matmul( cp.matmul(P,rhs_variable) , P )
    rhs_partial = nlg.partial_trace(rhs_permuted, [dimT, dim_TSS])
    rhs = cp.Constant(probQ2[q2]) * cp.kron(rhoT, rhs_partial)
    
    constraints.append( lhs - rhs == 0 )

# 5) NPA style constraint (see PhysRevLett.98.010401)

# The swap operator is
F_TTSS = nlg.permutation_matrix((0,1), (1,0), (dim_TT, dim_SS))

# The P matrix containing the information about the variables rho_TTSS is give by
P = []

for a1,q1 in indices_A1Q1:
    P_row = [dimT**2/(probQ1[q1]*probQ2[q2])*cp.trace(cp.matmul(F_TTSS,rho_TTSS[nlg.binarytoint([a1,q1,a2,q2])]))
             for a2,q2 in indices_A2Q2]
    P.append(P_row)

P = cp.bmat(P)

# Introduce new variables Q and R, symmetic matrices whose elements are < 1

# Q is dimA1*dimQ1 x dimA1*dimQ1
Q = cp.Variable((dim_A1Q1,dim_A1Q1),symmetric=True)

# R is dimA2*dimQ2 x dimA2*dimQ2
R = cp.Variable((dim_A2Q2,dim_A2Q2),symmetric=True)

# 5.1) The matrix M = [[Q,P],[P^T,R]] is PSD (NPA constraint)
M = cp.bmat([[Q,P],[P.T,R]])
constraints.append( M >> 0 )

# 5.2) Assumptions on projective nature of measurement on A and B

# Assumption over Q
for alpha in range(dim_A1Q1):
    for beta in range(dim_A1Q1):
        if alpha == beta:
            P_alpha = sum([P[alpha,gamma] for gamma in range(dim_A2Q2)])/dimQ2
            constraints.append( Q[alpha,beta] - P_alpha == 0 )
        elif abs(alpha-beta) == 1:
            constraints.append( Q[alpha,beta] == 0 )

# Additional constraints for off-diagonal part of Q
constraints.append( Q[0,2] + Q[0,3] - Q[0,0] == 0 )
constraints.append( Q[1,2] + Q[1,3] - Q[1,1] == 0 )
constraints.append( Q[0,2] + Q[1,2] - Q[2,2] == 0 )
constraints.append( Q[0,3] + Q[1,3] - Q[3,3] == 0 )

# Assumption over R
for alpha in range(dim_A2Q2):
    for beta in range(dim_A2Q2):
        if alpha == beta:
            P_beta = sum([P[gamma,beta] for gamma in range(dim_A1Q1)])/dimQ1
            constraints.append( R[alpha,beta] - P_beta == 0 )
        elif abs(alpha-beta) == 1:
            constraints.append( R[alpha,beta] == 0 )

# Additional constraints for off-diagonal part of R
constraints.append( R[0,2] + R[0,3] - R[0,0] == 0 )
constraints.append( R[1,2] + R[1,3] - R[1,1] == 0 )
constraints.append( R[0,2] + R[1,2] - R[2,2] == 0 )
constraints.append( R[0,3] + R[1,3] - R[3,3] == 0 )
            
# 5) PPT criterium
for i in map(nlg.binarytoint,indices_A1Q1A2Q2):
    constraints.append( nlg.partial_transpose(rho_TTSS[i],subs_TTSS,(0,0,1,1)) >> 0 )
    constraints.append( nlg.partial_transpose(rho_TTSS[i],subs_TTSS,(1,0,0,0)) >> 0 )
    constraints.append( nlg.partial_transpose(rho_TTSS[i],subs_TTSS,(0,1,0,0)) >> 0 )
    
# Write the problem
prob = cp.Problem(cp.Maximize(cp.real(object_function)), constraints)

Solve the problem

In [111]:
prob.solve(verbose=True,solver='MOSEK')



Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 74022           
  Cones                  : 0               
  Scalar variables       : 6292            
  Matrix variables       : 65              
  Integer variables      : 0               

Optimizer started.
GP based matrix reordering started.
GP based matrix reordering terminated.
Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 74022           
  Cones                  : 0               
  Scalar variables       : 6292            
  Matrix variables       : 65              
  Integer variables      : 0               

Optimizer  - threads                : 2               
Optimizer  - solved problem         : the primal      
Optimizer  - Constraints            

1.0000000095991102

In [114]:
R.value

array([[0.54210818, 0.        , 0.45789182, 0.08421635],
       [0.        , 0.45789182, 0.        , 0.45789182],
       [0.45789182, 0.        , 0.45789182, 0.        ],
       [0.08421635, 0.45789182, 0.        , 0.54210818]])

# Symmetry reduction of CHSH game $n_1 = 2$, $n_2 = 1$.

NOTE: The bound is trivial, and it should not be (for CHSH game, this level should give $0.875$). More constraints should be expected, not sure how to get them though! Probably from the permutation invariance of the state, I got two of them but cannot find others. It might be the case that we need to use operators obtained from partial traceing of orthogonal projectors...

In [4]:
# Subsystems (A1 Q1) (A1' Q1') (A2 Q2)
subs_A1Q1A1Q1A2Q2 = (dimA1,dimQ1,dimA1,dimQ1,dimA2,dimQ2)
indices_A1Q1A1Q1A2Q2 = nlg.indices_list(subs_A1Q1A1Q1A2Q2)
dim_A1Q1A1Q1A2Q2 = fc.reduce(mul, subs_A1Q1A1Q1A2Q2, 1)

# Subsystems Q1 (A1' Q1') (A2 Q2)
subs_Q1A1Q1A2Q2 = (dimQ1,dimA1,dimQ1,dimA2,dimQ2)
indices_Q1A1Q1A2Q2 = nlg.indices_list(subs_Q1A1Q1A2Q2)

# Subsystems (A1 Q1) (A1' Q1') Q2
subs_A1Q1A1Q1Q2 = (dimA1,dimQ1,dimA1,dimQ1,dimQ2)
indices_A1Q1A1Q1Q2 = nlg.indices_list(subs_A1Q1A1Q1Q2)

# Subsystems (A1 Q1) (A2 Q2)
subs_A1Q1A2Q2 = (dimA1,dimQ1,dimA2,dimQ2)
indices_A1Q1A2Q2 = nlg.indices_list(subs_A1Q1A2Q2)

# Subsystems (A1 Q1) or (A1' Q1')
subs_A1Q1 = (dimA1,dimQ1)
indices_A1Q1 = nlg.indices_list(subs_A1Q1)

# Subsystems (A2 Q2)
subs_A2Q2 = (dimA2,dimQ2)
indices_A2Q2 = nlg.indices_list(subs_A2Q2)

# State on subsystem T
rhoT = np.identity(dimT)/dimT

In [5]:
def seqtoint(sequence,dimension):
    for n,d in enumerate(dimension[1:]):
        sequence =[d*element for element in sequence[:n+1]] + list(sequence[n+1:])
    return sum(sequence)

StI = lambda seq : seqtoint(seq, subs_A1Q1A1Q1A2Q2)

The following is a test for the StI function created above.

In [6]:
#dic = {}
#
#for i in indices_A1Q1A1Q1A2Q2:
#    dic[''.join(list(map(str,i)))] = StI(i)
#    
#for key in sorted(dic.keys()):
#    print(key, dic[key])

## With CQ structure:

In [27]:
## VARIABLES

# The (sub-normalized) quantum states we optimize over
Lambda = []
Gamma = []
Sigma = []
Omega = []

for index in indices_A1Q1A1Q1A2Q2: 
    Lambda.append( cp.Variable((dimT,dimT),symmetric=True) )
    Gamma.append( cp.Variable((dimT,dimT),symmetric=True) )
    Sigma.append( cp.Variable((dimT,dimT),symmetric=True) )
    Omega.append( cp.Variable((dimT,dimT),symmetric=True) )
    
## OBJECTIVE FUNCTION

# The rule function V(a1,q1,a2,q2)
V = nlg.CHSH_rule_function_A1Q1A2Q2

# The object function is
obj_fun_components = []

for a1,q1,a2,q2 in indices_A1Q1A2Q2:
    indices_a1q1A1Q1a2q2 = [StI([a1,q1,a,q,a2,q2]) for a,q in indices_A1Q1]
    quantum_part = sum([Lambda[i]-Gamma[i] for i in indices_a1q1A1Q1a2q2])
    obj_fun_components.append( cp.Constant(V(a1,q1,a2,q2)) * cp.trace(quantum_part) )

object_function = cp.Constant(dimT**2) * sum(obj_fun_components)

## CONSTRAINTS
constraints = []

# 1) Positivity
for i in map(StI,indices_A1Q1A1Q1A2Q2):
    constraints.append( Lambda[i]+(Sigma[i]+Omega[i])/2. >> 0 )
    constraints.append( Gamma[i]-(Sigma[i]-Omega[i])/2. >> 0 )
    constraints.append( Lambda[i]-(Sigma[i]+Omega[i])/2. >> 0 )
    constraints.append( Gamma[i]+(Sigma[i]-Omega[i])/2. >> 0 )
    
    # From the above condition we get all the following ones and more
    #constraints.append( Lambda[i] >> 0 )
    #constraints.append( Gamma[i] >> 0 )
    #constraints.append( Sigma[i]+Lambda[i]+Gamma[i] >> 0 )
    #constraints.append( Omega[i]+Lambda[i]+Gamma[i] >> 0 )
                       
# 2) Normalization
norm_lambda_gamma_plus = cp.trace(sum([Lambda[i]+Gamma[i] for i in map(StI,indices_A1Q1A1Q1A2Q2)]))
norm_lambda_gamma_minus = cp.trace(sum([Lambda[i]-Gamma[i] for i in map(StI,indices_A1Q1A1Q1A2Q2)]))
norm_sigma = cp.trace(sum([Sigma[i] for i in map(StI,indices_A1Q1A1Q1A2Q2)]))
norm_omega = cp.trace(sum([Omega[i] for i in map(StI,indices_A1Q1A1Q1A2Q2)]))

constraints.append( norm_lambda_gamma_plus - 1 == 0 )
constraints.append( norm_lambda_gamma_minus + 1 >= 0 )
constraints.append( norm_lambda_gamma_minus - 1 <= 0 )
constraints.append( norm_sigma + 1 >= 0 )
constraints.append( norm_sigma - 1 <= 0 )
constraints.append( norm_omega + 1 >= 0 )
constraints.append( norm_omega - 1 <= 0 )

# From the above condition we get all the following ones and more, I think...
#norm_1 = cp.trace(sum([Lambda[i]+(Sigma[i]+Omega[i])/2. for i in map(StI,indices_A1Q1A1Q1A2Q2)]))
#norm_2 = cp.trace(sum([Gamma[i]-(Sigma[i]-Omega[i])/2. for i in map(StI,indices_A1Q1A1Q1A2Q2)]))
#norm_3 = cp.trace(sum([Lambda[i]-(Sigma[i]+Omega[i])/2. for i in map(StI,indices_A1Q1A1Q1A2Q2)]))
#norm_4 = cp.trace(sum([Gamma[i]+(Sigma[i]-Omega[i])/2. for i in map(StI,indices_A1Q1A1Q1A2Q2)]))
#constraints.append( norm_1 - 2 <= 0 )
#constraints.append( norm_2 - 2 <= 0 )
#constraints.append( norm_3 - 2 <= 0 )
#constraints.append( norm_4 - 2 <= 0 )

# 3) Permutation invariance
for a1,q1,a1p,q1p,a2,q2 in indices_A1Q1A1Q1A2Q2:
    index = StI([a1,q1,a1p,q1p,a2,q2])
    index_perm = StI([a1p,q1p,a1,q1,a2,q2])
    
    constraints.append( cp.trace(Sigma[index]-Sigma[index_perm]) == 0 )
    constraints.append( cp.trace((Lambda[index]+Gamma[index])-(Lambda[index_perm]+Gamma[index_perm])) == 0 )

# 4) First linear constraint
for q1,a1p,q1p,a2,q2 in indices_Q1A1Q1A2Q2:
    
    indices_A1q1a1pq1pa2q2 = [StI([a,q1,a1p,q1p,a2,q2]) for a in range(dimA1)]
    indices_A1Q1a1pq1pa2q2 = [StI([a,q ,a1p,q1p,a2,q2]) for a,q in indices_A1Q1]
    
    # lambda + gamma
    lhs_1 = sum([Lambda[i]+Gamma[i] for i in indices_A1q1a1pq1pa2q2])
    rhs_1 = cp.Constant(probQ1[q1]) * sum([Lambda[i]+Gamma[i] for i in indices_A1Q1a1pq1pa2q2])
    
    # lambda - gamma
    lhs_2 = sum([Lambda[i]-Gamma[i] for i in indices_A1q1a1pq1pa2q2])
    rhs_2 = cp.Constant(probQ1[q1]/dimT) * sum([Sigma[i] for i in indices_A1Q1a1pq1pa2q2])
    
    # sigma
    lhs_3 = sum([Sigma[i] for i in indices_A1q1a1pq1pa2q2])
    rhs_3 = cp.Constant(probQ1[q1]) * sum([Sigma[i] for i in indices_A1Q1a1pq1pa2q2])
    
    # omega
    lhs_4 = sum([Omega[i] for i in indices_A1q1a1pq1pa2q2])
    rhs_4 = cp.Constant(probQ1[q1]/dimT) * sum([Lambda[i]+Gamma[i] for i in indices_A1Q1a1pq1pa2q2])
    
    indices_a1pq1pA1q1a2q2 = [StI([a1p,q1p,a,q1,a2,q2]) for a in range(dimA1)]
    indices_a1pq1pA1Q1a2q2 = [StI([a1p,q1p,a,q ,a2,q2]) for a,q in indices_A1Q1]

    # lambda + gamma
    lhs_5 = sum([Lambda[i]+Gamma[i] for i in indices_a1pq1pA1q1a2q2])
    rhs_5 = cp.Constant(probQ1[q1]) * cp.trace(sum([Lambda[i]+Gamma[i] for i in indices_a1pq1pA1Q1a2q2])) * rhoT
    
    # lambda - gamma
    lhs_6 = sum([Lambda[i]-Gamma[i] for i in indices_a1pq1pA1q1a2q2])
    rhs_6 = cp.Constant(probQ1[q1]) * cp.trace(sum([Lambda[i]-Gamma[i] for i in indices_a1pq1pA1Q1a2q2])) * rhoT
    
    # sigma
    lhs_7 = sum([Sigma[i] for i in indices_a1pq1pA1q1a2q2])
    rhs_7 = cp.Constant(probQ1[q1]) * cp.trace(sum([Sigma[i] for i in indices_a1pq1pA1Q1a2q2])) * rhoT
    
    # omega
    lhs_8 = sum([Omega[i] for i in indices_a1pq1pA1q1a2q2])
    rhs_8 = cp.Constant(probQ1[q1]) * cp.trace(sum([Omega[i] for i in indices_a1pq1pA1Q1a2q2])) * rhoT
    
    constraints.append( lhs_1 - rhs_1 == 0 )
    constraints.append( lhs_2 - rhs_2 == 0 )
    constraints.append( lhs_3 - rhs_3 == 0 )
    constraints.append( lhs_4 - rhs_4 == 0 )
    constraints.append( lhs_5 - rhs_5 == 0 )
    constraints.append( lhs_6 - rhs_6 == 0 )
    constraints.append( lhs_7 - rhs_7 == 0 )
    constraints.append( lhs_8 - rhs_8 == 0 )
    
# 4) Second linear constraint
for a1,q1,a1p,q1p,q2 in indices_A1Q1A1Q1Q2:
    
    indices_a1q1a1pq1pA2q2 = [StI([a1,q1,a1p,q1p,a,q2]) for a in range(dimA2)]
    indices_a1q1a1pq1pA2Q2 = [StI([a1,q1,a1p,q1p,a,q ]) for a,q in indices_A2Q2]
    
    # lambda + gamma
    lhs_1 = sum([Lambda[i]+Gamma[i] for i in indices_a1q1a1pq1pA2q2])
    rhs_1 = cp.Constant(probQ2[q2]) * sum([Lambda[i]+Gamma[i] for i in indices_a1q1a1pq1pA2Q2])
    
    # lambda - gamma
    lhs_2 = sum([Lambda[i]-Gamma[i] for i in indices_a1q1a1pq1pA2q2])
    rhs_2 = cp.Constant(probQ2[q2]/dimT) * sum([Omega[i] for i in indices_a1q1a1pq1pA2Q2])
    
    # sigma
    lhs_3 = sum([Sigma[i] for i in indices_a1q1a1pq1pA2q2])
    rhs_3 = cp.Constant(probQ2[q2]/dimT) * sum([Lambda[i]+Gamma[i] for i in indices_a1q1a1pq1pA2Q2])
    
    # omega_T'
    lhs_4 = sum([Omega[i] for i in indices_a1q1a1pq1pA2q2])
    rhs_4 = cp.Constant(probQ2[q2]) * sum([Omega[i] for i in indices_a1q1a1pq1pA2Q2])
    
    constraints.append( lhs_1 - rhs_1 == 0 )
    constraints.append( lhs_2 - rhs_2 == 0 )
    constraints.append( lhs_3 - rhs_3 == 0 )
    constraints.append( lhs_4 - rhs_4 == 0 )
    
# 5) PPT conditions
for i in map(StI,indices_A1Q1A1Q1A2Q2):
    constraints.append( dimT * (Lambda[i]+Gamma[i]) - Sigma[i] >> 0 )
    constraints.append( dimT * (Lambda[i]+Gamma[i]) - Omega[i] >> 0 )
    constraints.append( dimT * Sigma[i] - (Lambda[i]-Gamma[i]) >> 0 )
    constraints.append( dimT * Omega[i] - (Lambda[i]-Gamma[i]) >> 0 )
    constraints.append( Lambda[i]-Gamma[i] >> 0 )
    constraints.append( Sigma[i] >> 0 )
    constraints.append( Omega[i] >> 0 )

# Write the problem
prob = cp.Problem(cp.Maximize(object_function), constraints)

In [28]:
prob.solve(verbose=True)



Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 4487            
  Cones                  : 0               
  Scalar variables       : 768             
  Matrix variables       : 704             
  Integer variables      : 0               

Optimizer started.
Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 4487            
  Cones                  : 0               
  Scalar variables       : 768             
  Matrix variables       : 704             
  Integer variables      : 0               

Optimizer  - threads                : 2               
Optimizer  - solved problem         : the primal      
Optimizer  - Constraints            : 3243
Optimizer  - Cones                  : 705
Optimizer  - Scalar variab

1.0000001473883653

## $I_{3322}$ inequality

NOTE: The $I_{3322}$ is given in terms of conditional probabilities, while our games is not. For this specific setting, we can unify the two by multiplying our result by 9, that is, the inverse of $\pi(q_1)\pi(q_2)$ for all $q_1, q_2$, since these distributions are uniform.

In [143]:
dimA1 = 2
dimA2 = 2
dimQ1 = 3
dimQ2 = 3

dimT = 2
dimS = 2

probQ1 = (1./3,1./3,1./3)
probQ2 = (1./3,1./3,1./3)

eta = 1.

In [144]:
# Subsystems A1 Q1 A2 Q2
subs_A1Q1A2Q2 = (dimA1,dimQ1,dimA2,dimQ2)
indices_A1Q1A2Q2 = nlg.indices_list(subs_A1Q1A2Q2)
dim_A1Q1A2Q2 = fc.reduce(mul, subs_A1Q1A2Q2, 1)

# Subsystems A2 Q1 Q2
subs_A2Q1Q2 = (dimA2,dimQ1,dimQ2)
indices_A2Q1Q2 = nlg.indices_list(subs_A2Q1Q2)

# Subsystems A1 Q1 Q2
subs_A1Q1Q2 = (dimA1,dimQ1,dimQ2)
indices_A1Q1Q2 = nlg.indices_list(subs_A1Q1Q2)

# Subsystems A1 Q1
subs_A1Q1 = (dimA1,dimQ1)
indices_A1Q1 = nlg.indices_list(subs_A1Q1)

# Subsystems A2 Q2
subs_A2Q2 = (dimA2,dimQ2)
indices_A2Q2 = nlg.indices_list(subs_A2Q2)

# Subsystems T hat(T) S hat(S)
subs_TTSS = (dimT,dimT,dimS,dimS)
indices_TTSS = nlg.indices_list(subs_TTSS)
dim_TTSS = fc.reduce(mul, subs_TTSS, 1)

# Additional dimensions
dim_TSS = dimT * dimS**2
dim_TT = dimT**2
dim_SS = dimS**2

# State on subsystem T
rhoT = np.identity(dimT)/dimT

Rule function and list-to-integer function: 

In [145]:
def I3322_rule_function(a1,q1,a2,q2,eta=1.):
    
    index = str(a1)+str(a2)+str(q1)+str(q2)

    I3322_rules = {"0000":1./3-1./(3*eta),
                   "0001":2./3-1./(3*eta),
                   "0002":1.-1./(3*eta),
                   "0010":1./3,
                   "0011":2./3,
                   "0012":-1.,
                   "0020":1./3,
                   "0021":-4./3,
                   "0100":-1./(3*eta),
                   "0101":-1./(3*eta),
                   "0102":-1./(3*eta),
                   "1000":-2./3,
                   "1001":-1./3,
                   "1010":-2./3,
                   "1011":-1./3,
                   "1020":-2./3,
                   "1021":-1./3}
    
    try:
        out = I3322_rules[index]
    except KeyError:
        out = 0
    
    return out

def seqtoint(sequence,dimension):
    for n,d in enumerate(dimension[1:]):
        sequence =[d*element for element in sequence[:n+1]] + list(sequence[n+1:])
    return sum(sequence)

# Here we define the function from sequence to integer for the I3322
StI = lambda seq : seqtoint(seq, subs_A1Q1A2Q2)

In [146]:
## VARIABLES

# The (sub-normalized) probability distributions we optimize over
p_A1Q1A2Q2 = cp.Variable(dim_A1Q1A2Q2,nonneg=True)
q_A1Q1A2Q2 = cp.Variable(dim_A1Q1A2Q2,nonneg=True)
r_A1Q1A2Q2 = cp.Variable(dim_A1Q1A2Q2)
s_A1Q1A2Q2 = cp.Variable(dim_A1Q1A2Q2)

## OBJECTIVE FUNCTION

# The rule function V(a1,q1,a2,q2)
V = I3322_rule_function

# The object function is
obj_fun_components = [V(*index,eta) * ( p_A1Q1A2Q2[StI(index)] - q_A1Q1A2Q2[StI(index)] )
                      for index in indices_A1Q1A2Q2]

object_function = cp.Constant(dimT**2) * sum(obj_fun_components)

## CONSTRAINTS

constraints = []
    
# 1) Positivity
constraints.append( p_A1Q1A2Q2 + q_A1Q1A2Q2 + r_A1Q1A2Q2 >= 0 )
constraints.append( p_A1Q1A2Q2 + q_A1Q1A2Q2 + s_A1Q1A2Q2 >= 0 )

# 2) Trace = 1
norm_p_q = sum([p_A1Q1A2Q2[i] + q_A1Q1A2Q2[i] for i in map(StI,indices_A1Q1A2Q2)])
norm_r = sum([r_A1Q1A2Q2[i] for i in map(StI,indices_A1Q1A2Q2)])
norm_s = sum([s_A1Q1A2Q2[i] for i in map(StI,indices_A1Q1A2Q2)])

constraints.append( norm_p_q - 1 == 0 )
constraints.append( norm_r + 1 >= 0 )
constraints.append( norm_r - 1 <= 0 )
constraints.append( norm_s + 1 >= 0 )
constraints.append( norm_s - 1 <= 0 )

# 2) First linear constraint
for a2,q1,q2 in indices_A2Q1Q2:
    indices_A1q1a2q2 = [StI([a,q1,a2,q2]) for a in range(dimA1)]
    indices_A1Q1a2q2 = [StI([a,q,a2,q2]) for a,q in indices_A1Q1]
    
    # p + q
    lhs_1 = sum([p_A1Q1A2Q2[i]+q_A1Q1A2Q2[i] for i in indices_A1q1a2q2])
    rhs_1 = cp.Constant(probQ1[q1]) * sum([p_A1Q1A2Q2[i]+q_A1Q1A2Q2[i] for i in indices_A1Q1a2q2])
    
    # p - q
    lhs_2 = sum([p_A1Q1A2Q2[i]-q_A1Q1A2Q2[i] for i in indices_A1q1a2q2])
    rhs_2 = cp.Constant(probQ1[q1]/dimT) * sum([r_A1Q1A2Q2[i] for i in indices_A1Q1a2q2])
    
    # r
    lhs_3 = sum([r_A1Q1A2Q2[i] for i in indices_A1q1a2q2])
    rhs_3 = cp.Constant(probQ1[q1]) * sum([r_A1Q1A2Q2[i] for i in indices_A1Q1a2q2])
    
    # s
    lhs_4 = sum([s_A1Q1A2Q2[i] for i in indices_A1q1a2q2])
    rhs_4 = cp.Constant(probQ1[q1]/dimT) * sum([p_A1Q1A2Q2[i]+q_A1Q1A2Q2[i] for i in indices_A1Q1a2q2])
    
    constraints.append( lhs_1 - rhs_1 == 0 )
    constraints.append( lhs_2 - rhs_2 == 0 )
    constraints.append( lhs_3 - rhs_3 == 0 )
    constraints.append( lhs_4 - rhs_4 == 0 )

# 3) Second linear constraint
for a1,q1,q2 in indices_A1Q1Q2:
    indices_a1q1A2q2 = [StI([a1,q1,a,q2]) for a in range(dimA2)]
    indices_a1q1A2Q2 = [StI([a1,q1,a,q]) for a,q in indices_A2Q2]
    
    # p + q
    lhs_1 = sum([p_A1Q1A2Q2[i]+q_A1Q1A2Q2[i] for i in indices_a1q1A2q2])
    rhs_1 = cp.Constant(probQ2[q2]) * sum([p_A1Q1A2Q2[i]+q_A1Q1A2Q2[i] for i in indices_a1q1A2Q2])
    
    # p - q
    lhs_2 = sum([p_A1Q1A2Q2[i]-q_A1Q1A2Q2[i] for i in indices_a1q1A2q2])
    rhs_2 = cp.Constant(probQ2[q2]/dimT) * sum([s_A1Q1A2Q2[i] for i in indices_a1q1A2Q2])
    
    # r
    lhs_3 = sum([r_A1Q1A2Q2[i] for i in indices_a1q1A2q2])
    rhs_3 = cp.Constant(probQ2[q2]/dimT) * sum([p_A1Q1A2Q2[i]+q_A1Q1A2Q2[i] for i in indices_a1q1A2Q2])
    
    # s
    lhs_4 = sum([s_A1Q1A2Q2[i] for i in indices_a1q1A2q2])
    rhs_4 = cp.Constant(probQ2[q2]) * sum([s_A1Q1A2Q2[i] for i in indices_a1q1A2Q2])
    
    constraints.append( lhs_1 - rhs_1 == 0 )
    constraints.append( lhs_2 - rhs_2 == 0 )
    constraints.append( lhs_3 - rhs_3 == 0 )
    constraints.append( lhs_4 - rhs_4 == 0 )
        
# 4) PPT conditions
constraints.append( p_A1Q1A2Q2 - q_A1Q1A2Q2 >= 0 )
constraints.append( r_A1Q1A2Q2 >= 0 )
constraints.append( s_A1Q1A2Q2 >= 0 )

# Write the problem
prob = cp.Problem(cp.Maximize(object_function), constraints)

In [147]:
I = prob.solve(verbose=True,solver='MOSEK')
print(I)
print(9*I)



Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : LO (linear optimization problem)
  Constraints            : 401             
  Cones                  : 0               
  Scalar variables       : 144             
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer started.
Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : LO (linear optimization problem)
  Constraints            : 401             
  Cones                  : 0               
  Scalar variables       : 144             
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer  - threads                : 2               
Optimizer  - solved problem         : the primal      
Optimizer  - Constraints            : 98
Optimizer  - Cones                  : 0
Optimizer  - Scalar variables     

In [148]:
## VARIABLES

# The (sub-normalized) states we optimize over
rho_TTSS = []
for i in map(StI,indices_A1Q1A2Q2):
    rho_TTSS.append( cp.Variable((dim_TTSS,dim_TTSS),symmetric=True) )

## OBJECTIVE FUNCTION

# The rule function is
rule_A1Q1A2Q2 = I3322_rule_function

# The swap operator is
F_TTSS = nlg.permutation_matrix((0,1), (1,0), (dim_TT, dim_SS))

# The object function is
obj_fun_components = [rule_A1Q1A2Q2(*index,eta) * cp.trace( cp.matmul(F_TTSS,rho_TTSS[StI(index)]) )
                      for index in indices_A1Q1A2Q2]

object_function = cp.Constant(dimT**2) * sum(obj_fun_components)

## CONSTRAINTS

constraints = []
    
# 2) rho_TTSS are (sub-normalized) quantum states
# 2a) trace of the sum is 1
constraints.append( sum([cp.trace(rho_TTSS[i]) for i in map(StI,indices_A1Q1A2Q2)]) - 1 == 0 )

# 2b) positive semidefinite matrices
for i in map(StI,indices_A1Q1A2Q2):
    constraints.append( rho_TTSS[i] >> 0 )
    
# 3) First linear constraint
for a2,q1,q2 in indices_A2Q1Q2:
    indices_A1q1a2q2 = [StI([a,q1,a2,q2]) for a in range(dimA1)]
    indices_A1Q1a2q2 = [StI([a,q,a2,q2]) for a,q in indices_A1Q1]
    
    lhs = sum([rho_TTSS[i] for i in indices_A1q1a2q2])
    
    rhs_variable = sum([rho_TTSS[i] for i in indices_A1Q1a2q2])
    rhs_partial = nlg.partial_trace(rhs_variable, [dimT, dim_TSS])
    rhs = cp.Constant(probQ1[q1]) * cp.kron(rhoT, rhs_partial)
    
    constraints.append( lhs - rhs == 0 )
    
# 4) Second linear constraint
P = nlg.permutation_matrix((0,1,2,3),(1,0,2,3),subs_TTSS)

for a1,q1,q2 in indices_A1Q1Q2:
    indices_a1q1A2q2 = [StI([a1,q1,a,q2]) for a in range(dimA2)]
    indices_a1q1A2Q2 = [StI([a1,q1,a,q]) for a,q in indices_A2Q2]
    
    lhs_variable = sum([rho_TTSS[i] for i in indices_a1q1A2q2])
    lhs = cp.matmul( cp.matmul(P,lhs_variable) , P )
    
    rhs_variable = sum([rho_TTSS[i] for i in indices_a1q1A2Q2])
    rhs_permuted = cp.matmul( cp.matmul(P,rhs_variable) , P )
    rhs_partial = nlg.partial_trace(rhs_permuted, [dimT, dim_TSS])
    rhs = cp.Constant(probQ2[q2]) * cp.kron(rhoT, rhs_partial)
    
    constraints.append( lhs - rhs == 0 )
    
# 5) PPT criterium
for i in map(StI,indices_A1Q1A2Q2):
    constraints.append( nlg.partial_transpose(rho_TTSS[i],subs_TTSS,(0,0,1,1)) >> 0 )
    constraints.append( nlg.partial_transpose(rho_TTSS[i],subs_TTSS,(1,0,0,0)) >> 0 )
    constraints.append( nlg.partial_transpose(rho_TTSS[i],subs_TTSS,(0,1,0,0)) >> 0 )
    
# Write the problem
prob = cp.Problem(cp.Maximize(object_function), constraints)

In [149]:
I = prob.solve(verbose=True,solver='MOSEK')
print(I)
print(9*I)



Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 46081           
  Cones                  : 0               
  Scalar variables       : 4896            
  Matrix variables       : 144             
  Integer variables      : 0               

Optimizer started.
Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 46081           
  Cones                  : 0               
  Scalar variables       : 4896            
  Matrix variables       : 144             
  Integer variables      : 0               

Optimizer  - threads                : 2               
Optimizer  - solved problem         : the primal      
Optimizer  - Constraints            : 38295
Optimizer  - Cones                  : 1
Optimizer  - Scalar variabl